# Задание по много поточности:

Вам необходимо проанализировать википедию на предмет того, какие слова в каждой из частей речи встречаются чаще. Вы хотите реализовать это в несколько потоков.

Запросы к википедии можно осуществлять с помощью библиотеки wikipedia. Для морфологического анализа используйте библиотеку pymorphy2. Чтобы разбить текст на слова можете воспользоваться функцией word_tokenize из библиотеки nltk.

Класс должен иметь функции, приведенные ниже (но может иметь и другие на ваше усмотрение).

### Часть 1

<b>Многопоточной реализация</b>

Задачи делятся на три типа:
<ul>
<li><i>Получение данных</i>:
<ol>
<li>Получение заголовков для страниц википедии - запускает по max_threads функций, которые асинхронно получают заголовки страниц.</li>
<li>Получение конкретных страниц - ждем, пока не появятся новые заголовки, которые не обработаны.
Когда появились - начинаем запрашивать в max_threads функциях конкретные страницы по заголовкам.</li>
</ol>
</li>
<li><i>Обработка данных</i>:
<ol>
<li>Ждем, пока не появятся новые необработанные страницы. Когда появляются, запускаем по max_threads функций для морфологического анализа слов.</li>
<li>Ждем пока не появились обработанные слова. Как только появляется новое слово, сразу же обновляем _stats.</li>
</ol>
</li>
<li><i>Сохранение данных</i>:
<ol>
        Раз в store_every обработанных слов вызывается асинхронно функция dump, которая сохраняет _stats.
</ol>
</li>
</ul>
<b>P. S.</b>

Комментарии специально запутанные, чтобы вы сами придумали архитектуру вызова потоков. Не бойтесь использовать Queue и daemon=True. Запрещается использовать threading.Lock / threading.RLock или другие блокировки.

In [3]:
import pymorphy2
import numpy as np
import wikipedia
import nltk

In [4]:
from queue import Queue
from collections import defaultdict
import threading

In [5]:
wikipedia.set_lang("ru")

In [6]:
def GetArticle():
    page_title = wikipedia.random(pages=1)
    return page_title

In [7]:
def GetWordsList(article):
    while True:
        try:
            page = wikipedia.page(title=article)
            tokens = nltk.word_tokenize(page.content)
            tokens = [token.lower() for token in tokens if token.isalpha()]
            break
        except:
            print("warning: couldn't get {}, retry".format(article))
            article = GetArticle()
    return tokens

In [ ]:
for i in range(10):
    try:
        title  = GetArticle()
        words = GetWordsList(title)[:10]
        print(words)
    except:
        print(title)

In [8]:
def thread_job_searcher(tasks_queue, articles_queue):
    while True:
        item = tasks_queue.get()
        if item is None:
            break
        article = GetArticle()
        articles_queue.put(article)
        tasks_queue.task_done()
    articles_queue.put(None)
        
def thread_job_proccesor_titles(articles_queue, words_queue):
    while True:
        item = articles_queue.get()
#         print("processing {}".format(item))
        if item is None:
            break
        words = GetWordsList(item)
        words_queue.put(words)
        articles_queue.task_done()
    words_queue.put(None)
    
def thread_job_proccesor_pages(words_queue, morph_queue):
    while True:
        item = words_queue.get()
        if item is None:
            break
        print("processing {}".format(item[:10]))
        morph = pymorphy2.MorphAnalyzer()
        for word in item:
            ans = morph.parse(word)[0].tag.POS
            morph_queue.put(ans)
        words_queue.task_done()
    morph_queue.put(None)
        
def thread_job_proccesor_words(morph_queue, stats):
    while True:
        item = morph_queue.get()
        if item is None:
            break
        print("processing {}".format(item))
        stats[item] += 1
        morph_queue.task_done()

In [30]:
num_threads = 5
count_articles = 10
tasks_queue = Queue()
articles_queue = Queue()
words_queue = Queue()
morph_queue = Queue()
stats = defaultdict(int)

for i in range(count_articles):
    tasks_queue.put(i)
    
for i in range(num_threads):
    tasks_queue.put(None)

In [31]:
threads = []
for i in range(num_threads):
    t = threading.Thread(target=thread_job_searcher, args=(tasks_queue, articles_queue))
    t.start()
    threads.append(t)
    t = threading.Thread(target=thread_job_proccesor_titles, args=(articles_queue, words_queue))
    t.start()
    threads.append(t)

    t = threading.Thread(target=thread_job_proccesor_pages, args=(words_queue, morph_queue))
    t.start()
    threads.append(t)
    t = threading.Thread(target=thread_job_proccesor_words, args=(morph_queue, stats))
    t.start()
    threads.append(t)


for t in threads:
    t.join()

processing ['jill', 'ann', 'sterkel', 'born', 'may', 'is', 'an', 'american', 'former', 'competition']
processing ['astrapogon', 'is', 'a', 'genus', 'of', 'cardinalfishes', 'native', 'to', 'the', 'western']
processing ['aapka', 'tv', 'is', 'an', 'telugu', 'and', 'hindi', 'news', 'and', 'social']
processing ['onibury', 'railway', 'station', 'was', 'a', 'station', 'in', 'onibury', 'shropshire', 'england']
processing ['arthur', 'whitelock', 'lemon', 'april', 'may', 'was', 'a', 'welsh', 'international', 'number']
processing ['angelo', 'paravisi', 'september', 'september', 'was', 'the', 'bishop', 'of', 'crema', 'from']
processing ['the', 'tornado', 'is', 'a', 'wooden', 'roller', 'coaster', 'located', 'at', 'adventureland']
processing ['the', 'kosovo', 'national', 'under', 'football', 'team', 'albanian', 'përfaqësuesja', 'e', 'futbollit']
processing ['eady', 'is', 'a', 'surname', 'notable', 'people', 'with', 'the', 'surname', 'include']
processing ['shaun', 'mark', 'bean', 'born', 'april', 'k

In [32]:
morph = pymorphy2.MorphAnalyzer()
morph.parse("cat")[0].tag.POS

In [ ]:
class WikiReader(object):
    """
    Класс для работы с википедией.
    Собирает статисткику по словам каждой части речи в статьях википедии.

    Parameters
    ----------
    morphs: list
        Части речи, которые хотим исследовать. Слова другой части речи не включаются в статистику.
    
    page_per_req: int
        Количество случайных названий страниц, запрашиваемых за один раз у википедии.
    
    max_threads: int
        Количество потоков, запускаемых другим потоком демоном (можно не использовать, если получится).
    
    max_words: int
        Количество слов для обработки.
    
    store_every: int
        Как часто сохранять данные на диск. Каждые store_every слов вызывается функция dump.
    
    store_path: str
        Куда сохранять данные.
    
    Attributes
    ----------
    _stats: <your code here>
        Структура данных (возможно встроенная), позволяющая хранить для каждой части речи список слов с их количеством.
        Необходимо, чтобы получение (изменение) статистики (текущего количества) для каждой пары
        <часть речи, слово> происходило за O(1).
            
    """
    def __init__(self,
                 morphs=[],
                 page_per_req=4,
                 max_threads_per_daemon=8,
                 max_words=10000,
                 store_every=1000,
                 store_path=""):
        self._stats = {}
    
    def run(self):
        <your code here>
    
    def dump(self, path=None):
        <your code here>
    
    def load(self, path=None):
        <your code here>
    pass